<a href="https://colab.research.google.com/github/achrisk/Dissertation/blob/main/ubCSbenchmark_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.io import loadmat
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt

from userDefFunc import my_function
from userDefFunc import f_logVar

from sklearn import svm
from sklearn.model_selection import cross_val_score

subList = ["P01", "P02", "P03", "P04","P05", "P06", "P07","P08", "P09", "P10"]

info = np.empty(8)
for sb in range(8):
    
    subID=subList[sb]

    dataSRC = "/media/andreas/Andreas/Dissertation/EEG data/" #Change the path according to your computer wbere the patient dataset is there.
    fnameWithPath=dataSRC+"parsed_"+subID+"T.mat"
    print(fnameWithPath)

    subDataFile = loadmat(fnameWithPath)
    RawEEGData = subDataFile["RawEEGData"]
    Labels = subDataFile["Labels"]
    sampRate = subDataFile["sampRate"]

    dim=RawEEGData.shape

    nOfTrials=dim[0] #getting the number of trials
   
    nOfChann=dim[1]  #getting the number of channels
   
    nOfSampsInTrial=dim[2] #getting the number of samples per trial
    

    order=4
    muBand=([8, 12]/sampRate)*2
    muBand=muBand[0]
    betaBand=([16, 24]/sampRate)*2
    betaBand=betaBand[0]

    mu_B, mu_A = signal.butter(order, muBand, 'bandpass', analog=False)
    beta_B, beta_A = signal.butter(order, betaBand, 'bandpass', analog=False)

    muRawEEGData=np.empty((nOfTrials, nOfChann, nOfSampsInTrial))
    betaRawEEGData=np.empty((nOfTrials, nOfChann, nOfSampsInTrial))

    for trlIndex in range(nOfTrials):

        sig=RawEEGData[trlIndex,:,:]
    
        mu_temp=signal.lfilter(mu_B, mu_A, sig,1)
        beta_temp=signal.lfilter(beta_B, beta_A, sig,1)

        muRawEEGData[trlIndex,:,:]= mu_temp #altering the dimensions
        betaRawEEGData[trlIndex,:,:]= beta_temp #altering the dimensions
        

    labelsCls1 = np.where(Labels == 1)
    labelsCls1=labelsCls1[0]
    labelsCls2 = np.where(Labels == 2)
    labelsCls2=labelsCls2[0]

    muRawEEGDataCls1=muRawEEGData[labelsCls1,:,:] #mu band data from class 1 trials
    muRawEEGDataCls2=muRawEEGData[labelsCls2,:,:] #mu band data from class 2 trials

    betaRawEEGDataCls1=betaRawEEGData[labelsCls1,:,:] #beta band data from class 1 trials
    betaRawEEGDataCls2=betaRawEEGData[labelsCls2,:,:] #beta band data from class 2 trials

    

    muRawEEGDataCls1=np.swapaxes(muRawEEGDataCls1,0,1)
    muRawEEGDataCls2=np.swapaxes(muRawEEGDataCls2,0,1)
    betaRawEEGDataCls1=np.swapaxes(betaRawEEGDataCls1,0,1)
    betaRawEEGDataCls2=np.swapaxes(betaRawEEGDataCls2,0,1)

    

    cspMuRawEEGDataCls1 = np.reshape(muRawEEGDataCls1, (nOfChann, int(nOfTrials/2)*nOfSampsInTrial))
    cspMuRawEEGDataCls2 = np.reshape(muRawEEGDataCls2, (nOfChann, int(nOfTrials/2)*nOfSampsInTrial))
    cspBetaRawEEGDataCls1 = np.reshape(betaRawEEGDataCls1, (nOfChann, int(nOfTrials/2)*nOfSampsInTrial))
    cspBetaRawEEGDataCls2 = np.reshape(betaRawEEGDataCls2, (nOfChann, int(nOfTrials/2)*nOfSampsInTrial))

    

    wCSP_mu = my_function(cspMuRawEEGDataCls1,cspMuRawEEGDataCls2)
    wCSP_beta = my_function(cspBetaRawEEGDataCls1,cspBetaRawEEGDataCls2)

    
    feat = np.empty((nOfTrials,4))

    for trlIndex in range(nOfTrials):

        muTemp=muRawEEGData[trlIndex,:,:]
        betaTemp=betaRawEEGData[trlIndex,:,:]

        mu_temp = np.matmul(wCSP_mu,muTemp);  #calculating the Z matrix for mu band
        beta_temp = np.matmul(wCSP_beta,betaTemp);   #calculating the Z matrix for beta band

        logVarMu=f_logVar(mu_temp);      #calculating the logvariance for Mu
        logVarBeta=f_logVar(beta_temp);  #calculating the logvariance for Beta

        feat[trlIndex, :]=[logVarMu[0],  logVarMu[len(logVarMu)-1],  logVarBeta[0],  logVarBeta[len(logVarBeta)-1]]

    Train_X = feat
    Train_Y = Labels[:,0]

    clf = svm.SVC(kernel='linear', C=1)

    svmMdl = clf.fit(Train_X, Train_Y)

    Train_Y_predicted = svmMdl.predict(Train_X)
    from sklearn.metrics import accuracy_score
    trainAcc = accuracy_score(Train_Y_predicted, Train_Y)

    scores = cross_val_score(clf, Train_X, Train_Y, cv=10)
    meanCVacc = np.mean(scores)
    
    info[sb]=meanCVacc

    import pickle

    outputFilename="trainedModel"+subID
    with open(outputFilename, 'wb') as f:
        pickle.dump([svmMdl, wCSP_mu, wCSP_beta, muBand, betaBand], f)

print(info)
print("meanCVaccAllSub", np.mean(info))